In [29]:
#%% prepare_usersoil_spaw.py
import pandas as pd
import numpy as np
import math


def compute_hydgrp(T_SAND,S_SAND):
    T_Y = 0.0 #T_Y:上层土壤粒径
    S_Y = 0.0 #S_Y:下层土壤粒径
    T_X = 0.0 #T_X:上层土壤下渗率
    S_X = 0.0 #S_X:下层土壤下渗率
    HYDGRP = 'A' #HYDGRP:土壤水文组
    if T_SAND==0.0:
        T_Y = 0.01
    elif T_SAND==100.0:
        T_Y = 0.3
    else:
        T_Y = T_SAND/10*0.03+0.02
    
    if S_SAND==0.0:
        S_Y = 0.01
    elif S_SAND==100.0:
        S_Y = 0.3
    else:
        S_Y = S_SAND/10*0.03+0.02
    
    T_X = (20*T_Y)**1.8
    S_X = (20*S_Y)**1.8


    MIN_X = min(T_X,S_X)
    print("T_X={}; S_X={}; min_x={}".format(T_X,S_X,MIN_X))
    if MIN_X>=0 and MIN_X<=1.3:
        HYDGRP = 'D'
    elif MIN_X>1.3 and MIN_X<=3.8:
        HYDGRP = 'C'
    elif MIN_X>3.8 and MIN_X<=7.6:
        HYDGRP = 'B'
    elif MIN_X>7.6:
        HYDGRP = 'A'
    return HYDGRP

def compute_usle_k(SAND,SILT,CLAY,OC):
    f_csand = 0.2+0.3*math.exp(-0.256*SAND*(1-SILT/100.0))
    f_cl_si = (SILT/(CLAY+SILT))**0.3
    f_orgc = 1.0-0.25*OC/(OC+math.exp(3.72-2.95*OC))
    f_hisand = 1.0-0.7*(1-SAND/100.0)/((1.0-SAND/100.0)+math.exp(-5.51+22.9*(1-SAND/100.0)))
    print('SAND={},SILT={},CLAY={},OC={}'.format(SAND,SILT,CLAY,OC))
    print('f_csand={},f_cl_si={},f_orgc={},f_hisand={}'.format(f_csand,f_cl_si,f_orgc,f_hisand))
    return f_csand*f_cl_si*f_orgc*f_hisand

def compute_sol_alb(OC):# OC=Organic Matter
    return 0.2313*math.exp(-1.9448*OC)

In [30]:
# *工程路径：D:\WorkSpace\HydroModel\SWATPlus\SWATPreprocesPyProject
#!程序运行前准备:HWSD_DATA.xlsx, 中国区HWSD土壤类型名称及对应土组.xlsx,target_soil.dbf.xlsx,usersoil.xlsx
#!HWSD_DATA.xlsx从hwsd_data_loc位置下HWSD.mdb数据库导出
#!中国区HWSD土壤类型名称及对应土组.xlsx自己准备或者从网上下载，保存至hwsd_data_loc位置下
#!target_soil.dbf.xlsx从GIS软件（ArcGIS/ArcGIS pro/QGIS）导出的研究区域土壤数据分布属性表:右键打开属性表，然后导出为target_soil.dbf,用excel打开另存为EXCEL格式（target_soil.dbf.xlsx文件）,保存至workspace空间中
#!usersoil.xlsx：从SWAT2012.mdb中导出usersoil为usersoil.xlsx,保存至workspace空间中

data_name = 'Xunhua_data'

#! 设置工作空间和HWSD数据存储空间：这部分设置完再运行程序
workspace = f'../SWATPlusData/{data_name}/'
# workspace = 'D:/WorkSpace/HydroModel/SWATPlus/Tangnaihai_data/'
# workspace = 'SWATPlusData/ZhenAn_data/'
# workspace = 'SWATPlusData/Zhangjiashan_data/'
# hwsd_data_loc = 'D:/WorkSpace/ArcGis/Harmonized World Soil Database v 1.2/'

standard_data_loc = '../data/'

# 导入从HWSD.mdb中导出的HWSD_DATA数据
HWSD_DATA = pd.read_excel(standard_data_loc+"HWSD_DATA.xlsx", sheet_name='HWSD_DATA')
HWSD_CHINA_INFO = pd.read_excel(standard_data_loc+'中国区HWSD土壤类型名称及对应土组.xlsx',sheet_name='D_SYMBOL90')
# target_soil = pd.read_excel(workspace+"target_soil.dbf.xlsx",sheet_name='target_soil')
target_soil = pd.read_excel(workspace+"soil.dbf.xlsx",sheet_name='soil')
usersoil = pd.read_excel(standard_data_loc+"usersoil.xlsx",sheet_name='usersoil')

In [31]:
# 从HWSD_DATA数据中提取与soil(or target_soil)对应的土壤参数信息：HWSD_DATA中MU_GLOBAL对应于soil(target_soil)中的Value
EXTRACTED_HWSD_DATA = HWSD_DATA.loc[HWSD_DATA['MU_GLOBAL'].isin(target_soil['Value'])].reset_index()
EXTRACTED_HWSD_DATA.to_csv('temp.csv')
# 将提取出来的EXTRACTED_HWSD_DATA和target_soil合并，获取SWAT土壤属性计算参数表
SOL_TABLE = pd.concat([target_soil,EXTRACTED_HWSD_DATA],axis=1)

SU_SYM90_SOL_GRP = dict(zip(HWSD_CHINA_INFO['SYMBOL'],HWSD_CHINA_INFO['土组']))#SYMBOL对应于HWSD_DATA中SU_SYM90

for key in SU_SYM90_SOL_GRP:
    if pd.isna(SU_SYM90_SOL_GRP[key]):
        # print(type(SU_SYM90_SOL_GRP[key]))
        # print(SU_SYM90_SOL_GRP[key])
        SU_SYM90_SOL_GRP[key]=key
print(SU_SYM90_SOL_GRP)

SOL_TABLE.insert(2,'SOIL_GROUP',  SOL_TABLE['SU_SYM90'].map(SU_SYM90_SOL_GRP))
SOL_TABLE.insert(2,'RASTER_RATIO',  SOL_TABLE["Count"] / SOL_TABLE.groupby(["SOIL_GROUP"])["Count"].transform("sum"))

SOL_TABLE.to_csv(workspace+'SOL_TABLE.csv',index=False,encoding='utf-8')

{'FLe': '冲积土（FLUVISOLS）', 'FLc': '冲积土（FLUVISOLS）', 'FLu': '冲积土（FLUVISOLS）', 'FLs': '冲积土（FLUVISOLS）', 'GLe': '潜育土（GLEYSOLS）', 'GLk': '潜育土（GLEYSOLS）', 'GLm': '潜育土（GLEYSOLS）', 'GLt': '潜育土（GLEYSOLS）', 'GLi': '潜育土（GLEYSOLS）', 'ACh': '低活性强酸土（ACRISOLS）', 'ACf': '低活性强酸土（ACRISOLS）', 'ACu': '低活性强酸土（ACRISOLS）', 'ACp': '低活性强酸土（ACRISOLS）', 'ALh': '高活性强酸土（ALISOLS）', 'ALf': '高活性强酸土（ALISOLS）', 'ALp': '高活性强酸土（ALISOLS）', 'ALj': '高活性强酸土（ALISOLS）', 'ANh': '火山灰土（ANDOSOLS）', 'ANu': '火山灰土（ANDOSOLS）', 'ARh': '砂性土（ARENOSOLS）', 'ARb': '砂性土（ARENOSOLS）', 'ARl': '砂性土（ARENOSOLS）', 'ARa': '砂性土（ARENOSOLS）', 'ARc': '砂性土（ARENOSOLS）', 'AT': '人为土（ANTHROSOLS）', 'ATa': '人为土（ANTHROSOLS）', 'ATc': '人为土（ANTHROSOLS）', 'ATf': '人为土（ANTHROSOLS）', 'CHh': '黑钙土（CHERNOZEMS）', 'CHk': '黑钙土（CHERNOZEMS）', 'CHl': '黑钙土（CHERNOZEMS）', 'CHg': '黑钙土（CHERNOZEMS）', 'CLh': '钙积土（CALCISOLS）', 'CLl': '钙积土（CALCISOLS）', 'CLp': '钙积土（CALCISOLS）', 'CMe': '雏形土（CAMBISOLS）', 'CMd': '雏形土（CAMBISOLS）', 'CMu': '雏形土（CAMBISOLS）', 'CMc': '雏形土（CAMBISOLS）', 'CMx': '雏形

In [32]:
# group by SOIL_GROUP and get the index of the row with the max value in RASTER_RATIO for each group
idx = SOL_TABLE.groupby('SOIL_GROUP')['RASTER_RATIO'].idxmax()

# 重分类（select the rows with the indices obtained above）
SOL_TABLE_RECLASS = SOL_TABLE.loc[idx]
SOL_TABLE_RECLASS.reset_index(drop=True,inplace=True)

print(SOL_TABLE_RECLASS[["MU_GLOBAL","T_OC"]])

SOLGRP_RECLASSVAL = dict(zip(SOL_TABLE_RECLASS['SOIL_GROUP'],SOL_TABLE_RECLASS['Value']))

RECLASS_TABLE = pd.DataFrame()
FROM_ = []
TO = []
OUT = []
MAPPING = []

for i in range(SOL_TABLE.shape[0]):
    FROM_.append(SOL_TABLE['Value'][i])
    TO.append(SOL_TABLE['Value'][i])
    OUT.append(SOLGRP_RECLASSVAL[SOL_TABLE['SOIL_GROUP'][i]])
    MAPPING.append('ValueToValue')

RECLASS_TABLE.insert(0,'FROM_',FROM_)
RECLASS_TABLE.insert(1,'TO',TO)
RECLASS_TABLE.insert(2,'OUT',OUT)
RECLASS_TABLE.insert(3,'MAPPING',MAPPING)

# 导出重分类表格（用于输入ArcGIS pro-> Reclassify实现重分类）
# Svae RECLASS_TABLE data to arcgis table format


RECLASS_TABLE.to_excel(workspace+'RECLASS_TABLE.xlsx',index=None)


    MU_GLOBAL   T_OC
0       11930    NaN
1       11926    NaN
2       11933    NaN
3       11925    NaN
4       11927    NaN
5       11698   1.12
6       11341   0.60
7       11551  39.40
8       11158   1.20
9       11539   1.65
10      11103   1.60
11      11400   0.75
12      11555   0.49
13      11296   0.41
14      11353   0.43
15      11705   1.41
16      11256   0.46
17      11719   2.02
18      11909   0.74
19      11404   1.95
20      11108   1.64


In [33]:
#!处理不常见土壤类型SYMBOL：包括DS=沙丘流沙;ST=盐场、盐滩；GG=冰川；ND=无数据；FP=鱼塘；IS=岛屿
#!处理方式为用这些土壤类型附近占比最大的土壤类型的参数进行代理，名称不需要变，需要人工判断
# 存储需要替换为周围土壤的土壤的MU_GLOBAL
replace_mu_global = []

for i in range(SOL_TABLE_RECLASS.shape[0]):
    print(SOL_TABLE_RECLASS.loc[i,'SU_SYM90'])
    if SOL_TABLE_RECLASS.loc[i,'SU_SYM90'] == 'DS':
        replaced_mu_global = int(input('请输入替换'+SOL_TABLE_RECLASS.loc[i,'SU_SYM90']+'(MU_GLOBAL='+str(SOL_TABLE_RECLASS.loc[i,'MU_GLOBAL'])+')的MU_GLOBAL：'))
        replace_mu_global.append(str(SOL_TABLE_RECLASS.loc[i,'MU_GLOBAL']))
        replaced_data = HWSD_DATA.loc[HWSD_DATA['MU_GLOBAL']==replaced_mu_global].reset_index(drop=True)
        SOL_TABLE_RECLASS.loc[i,'REF_DEPTH'] = replaced_data.loc[0,'REF_DEPTH']
        SOL_TABLE_RECLASS.loc[i,'T_GRAVEL'] = replaced_data.loc[0,'T_GRAVEL']
        SOL_TABLE_RECLASS.loc[i,'T_SAND'] = replaced_data.loc[0,'T_SAND']
        SOL_TABLE_RECLASS.loc[i,'T_SILT'] = replaced_data.loc[0,'T_SILT']
        SOL_TABLE_RECLASS.loc[i,'T_CLAY'] = replaced_data.loc[0,'T_CLAY']
        SOL_TABLE_RECLASS.loc[i,'T_OC'] = replaced_data.loc[0,'T_OC']
        SOL_TABLE_RECLASS.loc[i,'T_PH_H2O'] = replaced_data.loc[0,'T_PH_H2O']
        SOL_TABLE_RECLASS.loc[i,'T_CACO3'] = replaced_data.loc[0,'T_CACO3']
        SOL_TABLE_RECLASS.loc[i,'T_ECE'] = replaced_data.loc[0,'T_ECE']
        SOL_TABLE_RECLASS.loc[i,'S_GRAVEL'] = replaced_data.loc[0,'S_GRAVEL']
        SOL_TABLE_RECLASS.loc[i,'S_SAND'] = replaced_data.loc[0,'S_SAND']
        SOL_TABLE_RECLASS.loc[i,'S_SILT'] = replaced_data.loc[0,'S_SILT']
        SOL_TABLE_RECLASS.loc[i,'S_CLAY'] = replaced_data.loc[0,'S_CLAY']
        SOL_TABLE_RECLASS.loc[i,'S_OC'] = replaced_data.loc[0,'S_OC']
        SOL_TABLE_RECLASS.loc[i,'S_CACO3'] = replaced_data.loc[0,'S_CACO3']
        SOL_TABLE_RECLASS.loc[i,'S_PH_H2O'] = replaced_data.loc[0,'S_PH_H2O']
    elif SOL_TABLE_RECLASS.loc[i,'SU_SYM90'] == 'ST':
        replaced_mu_global = int(input('请输入替换'+SOL_TABLE_RECLASS.loc[i,'SU_SYM90']+'(MU_GLOBAL='+str(SOL_TABLE_RECLASS.loc[i,'MU_GLOBAL'])+')的MU_GLOBAL：'))
        replaced_data = HWSD_DATA.loc[HWSD_DATA['MU_GLOBAL']==replaced_mu_global].reset_index(drop=True)
        replace_mu_global.append(str(SOL_TABLE_RECLASS.loc[i,'MU_GLOBAL']))
        print(replaced_data.loc[0,"T_OC"])
        SOL_TABLE_RECLASS.loc[i,'REF_DEPTH'] = replaced_data.loc[0,'REF_DEPTH']
        SOL_TABLE_RECLASS.loc[i,'T_GRAVEL'] = replaced_data.loc[0,'T_GRAVEL']
        SOL_TABLE_RECLASS.loc[i,'T_SAND'] = replaced_data.loc[0,'T_SAND']
        SOL_TABLE_RECLASS.loc[i,'T_SILT'] = replaced_data.loc[0,'T_SILT']
        SOL_TABLE_RECLASS.loc[i,'T_CLAY'] = replaced_data.loc[0,'T_CLAY']
        SOL_TABLE_RECLASS.loc[i,'T_OC'] = replaced_data.loc[0,'T_OC']
        SOL_TABLE_RECLASS.loc[i,'T_PH_H2O'] = replaced_data.loc[0,'T_PH_H2O']
        SOL_TABLE_RECLASS.loc[i,'T_CACO3'] = replaced_data.loc[0,'T_CACO3']
        SOL_TABLE_RECLASS.loc[i,'T_ECE'] = replaced_data.loc[0,'T_ECE']
        SOL_TABLE_RECLASS.loc[i,'S_GRAVEL'] = replaced_data.loc[0,'S_GRAVEL']
        SOL_TABLE_RECLASS.loc[i,'S_SAND'] = replaced_data.loc[0,'S_SAND']
        SOL_TABLE_RECLASS.loc[i,'S_SILT'] = replaced_data.loc[0,'S_SILT']
        SOL_TABLE_RECLASS.loc[i,'S_CLAY'] = replaced_data.loc[0,'S_CLAY']
        SOL_TABLE_RECLASS.loc[i,'S_OC'] = replaced_data.loc[0,'S_OC']
        SOL_TABLE_RECLASS.loc[i,'S_CACO3'] = replaced_data.loc[0,'S_CACO3']
        SOL_TABLE_RECLASS.loc[i,'S_PH_H2O'] = replaced_data.loc[0,'S_PH_H2O']
    elif SOL_TABLE_RECLASS.loc[i,'SU_SYM90'] == 'GG':
        replaced_mu_global = int(input('请输入替换'+SOL_TABLE_RECLASS.loc[i,'SU_SYM90']+'(MU_GLOBAL='+str(SOL_TABLE_RECLASS.loc[i,'MU_GLOBAL'])+')的MU_GLOBAL：'))
        replace_mu_global.append(str(SOL_TABLE_RECLASS.loc[i,'MU_GLOBAL']))
        replaced_data = HWSD_DATA.loc[HWSD_DATA['MU_GLOBAL']==replaced_mu_global].reset_index(drop=True)
        # print(replaced_data.loc[0,'REF_DEPTH'])
        SOL_TABLE_RECLASS.loc[i,'REF_DEPTH'] = replaced_data.loc[0,'REF_DEPTH']
        SOL_TABLE_RECLASS.loc[i,'T_GRAVEL'] = replaced_data.loc[0,'T_GRAVEL']
        SOL_TABLE_RECLASS.loc[i,'T_SAND'] = replaced_data.loc[0,'T_SAND']
        SOL_TABLE_RECLASS.loc[i,'T_SILT'] = replaced_data.loc[0,'T_SILT']
        SOL_TABLE_RECLASS.loc[i,'T_CLAY'] = replaced_data.loc[0,'T_CLAY']
        SOL_TABLE_RECLASS.loc[i,'T_OC'] = replaced_data.loc[0,'T_OC']
        SOL_TABLE_RECLASS.loc[i,'T_PH_H2O'] = replaced_data.loc[0,'T_PH_H2O']
        SOL_TABLE_RECLASS.loc[i,'T_CACO3'] = replaced_data.loc[0,'T_CACO3']
        SOL_TABLE_RECLASS.loc[i,'T_ECE'] = replaced_data.loc[0,'T_ECE']
        SOL_TABLE_RECLASS.loc[i,'S_GRAVEL'] = replaced_data.loc[0,'S_GRAVEL']
        SOL_TABLE_RECLASS.loc[i,'S_SAND'] = replaced_data.loc[0,'S_SAND']
        SOL_TABLE_RECLASS.loc[i,'S_SILT'] = replaced_data.loc[0,'S_SILT']
        SOL_TABLE_RECLASS.loc[i,'S_CLAY'] = replaced_data.loc[0,'S_CLAY']
        SOL_TABLE_RECLASS.loc[i,'S_OC'] = replaced_data.loc[0,'S_OC']
        SOL_TABLE_RECLASS.loc[i,'S_CACO3'] = replaced_data.loc[0,'S_CACO3']
        SOL_TABLE_RECLASS.loc[i,'S_PH_H2O'] = replaced_data.loc[0,'S_PH_H2O']
    elif SOL_TABLE_RECLASS.loc[i,'SU_SYM90'] == 'ND':
        replaced_mu_global = int(input('请输入替换'+SOL_TABLE_RECLASS.loc[i,'SU_SYM90']+'(MU_GLOBAL='+str(SOL_TABLE_RECLASS.loc[i,'MU_GLOBAL'])+')的MU_GLOBAL：'))
        replace_mu_global.append(str(SOL_TABLE_RECLASS.loc[i,'MU_GLOBAL']))
        replaced_data = HWSD_DATA.loc[HWSD_DATA['MU_GLOBAL']==replaced_mu_global].reset_index(drop=True)
        SOL_TABLE_RECLASS.loc[i,'REF_DEPTH'] = replaced_data.loc[0,'REF_DEPTH']
        SOL_TABLE_RECLASS.loc[i,'T_GRAVEL'] = replaced_data.loc[0,'T_GRAVEL']
        SOL_TABLE_RECLASS.loc[i,'T_SAND'] = replaced_data.loc[0,'T_SAND']
        SOL_TABLE_RECLASS.loc[i,'T_SILT'] = replaced_data.loc[0,'T_SILT']
        SOL_TABLE_RECLASS.loc[i,'T_CLAY'] = replaced_data.loc[0,'T_CLAY']
        SOL_TABLE_RECLASS.loc[i,'T_OC'] = replaced_data.loc[0,'T_OC']
        SOL_TABLE_RECLASS.loc[i,'T_PH_H2O'] = replaced_data.loc[0,'T_PH_H2O']
        SOL_TABLE_RECLASS.loc[i,'T_CACO3'] = replaced_data.loc[0,'T_CACO3']
        SOL_TABLE_RECLASS.loc[i,'T_ECE'] = replaced_data.loc[0,'T_ECE']
        SOL_TABLE_RECLASS.loc[i,'S_GRAVEL'] = replaced_data.loc[0,'S_GRAVEL']
        SOL_TABLE_RECLASS.loc[i,'S_SAND'] = replaced_data.loc[0,'S_SAND']
        SOL_TABLE_RECLASS.loc[i,'S_SILT'] = replaced_data.loc[0,'S_SILT']
        SOL_TABLE_RECLASS.loc[i,'S_CLAY'] = replaced_data.loc[0,'S_CLAY']
        SOL_TABLE_RECLASS.loc[i,'S_OC'] = replaced_data.loc[0,'S_OC']
        SOL_TABLE_RECLASS.loc[i,'S_CACO3'] = replaced_data.loc[0,'S_CACO3']
        SOL_TABLE_RECLASS.loc[i,'S_PH_H2O'] = replaced_data.loc[0,'S_PH_H2O']
    elif SOL_TABLE_RECLASS.loc[i,'SU_SYM90'] == 'FP':
        replaced_mu_global = int(input('请输入替换'+SOL_TABLE_RECLASS.loc[i,'SU_SYM90']+'(MU_GLOBAL='+str(SOL_TABLE_RECLASS.loc[i,'MU_GLOBAL'])+')的MU_GLOBAL：'))
        replace_mu_global.append(str(SOL_TABLE_RECLASS.loc[i,'MU_GLOBAL']))
        replaced_data = HWSD_DATA.loc[HWSD_DATA['MU_GLOBAL']==replaced_mu_global].reset_index(drop=True)
        SOL_TABLE_RECLASS.loc[i,'REF_DEPTH'] = replaced_data.loc[0,'REF_DEPTH']
        SOL_TABLE_RECLASS.loc[i,'T_GRAVEL'] = replaced_data.loc[0,'T_GRAVEL']
        SOL_TABLE_RECLASS.loc[i,'T_SAND'] = replaced_data.loc[0,'T_SAND']
        SOL_TABLE_RECLASS.loc[i,'T_SILT'] = replaced_data.loc[0,'T_SILT']
        SOL_TABLE_RECLASS.loc[i,'T_CLAY'] = replaced_data.loc[0,'T_CLAY']
        SOL_TABLE_RECLASS.loc[i,'T_OC'] = replaced_data.loc[0,'T_OC']
        SOL_TABLE_RECLASS.loc[i,'T_PH_H2O'] = replaced_data.loc[0,'T_PH_H2O']
        SOL_TABLE_RECLASS.loc[i,'T_CACO3'] = replaced_data.loc[0,'T_CACO3']
        SOL_TABLE_RECLASS.loc[i,'T_ECE'] = replaced_data.loc[0,'T_ECE']
        SOL_TABLE_RECLASS.loc[i,'S_GRAVEL'] = replaced_data.loc[0,'S_GRAVEL']
        SOL_TABLE_RECLASS.loc[i,'S_SAND'] = replaced_data.loc[0,'S_SAND']
        SOL_TABLE_RECLASS.loc[i,'S_SILT'] = replaced_data.loc[0,'S_SILT']
        SOL_TABLE_RECLASS.loc[i,'S_CLAY'] = replaced_data.loc[0,'S_CLAY']
        SOL_TABLE_RECLASS.loc[i,'S_OC'] = replaced_data.loc[0,'S_OC']
        SOL_TABLE_RECLASS.loc[i,'S_CACO3'] = replaced_data.loc[0,'S_CACO3']
        SOL_TABLE_RECLASS.loc[i,'S_PH_H2O'] = replaced_data.loc[0,'S_PH_H2O']
    elif SOL_TABLE_RECLASS.loc[i,'SU_SYM90'] == 'IS':
        replaced_mu_global = int(input('请输入替换'+SOL_TABLE_RECLASS.loc[i,'SU_SYM90']+'(MU_GLOBAL='+str(SOL_TABLE_RECLASS.loc[i,'MU_GLOBAL'])+')的MU_GLOBAL：'))
        replace_mu_global.append(str(SOL_TABLE_RECLASS.loc[i,'MU_GLOBAL']))
        replaced_data = HWSD_DATA.loc[HWSD_DATA['MU_GLOBAL']==replaced_mu_global].reset_index(drop=True)
        SOL_TABLE_RECLASS.loc[i,'REF_DEPTH'] = replaced_data.loc[0,'REF_DEPTH']
        SOL_TABLE_RECLASS.loc[i,'T_GRAVEL'] = replaced_data.loc[0,'T_GRAVEL']
        SOL_TABLE_RECLASS.loc[i,'T_SAND'] = replaced_data.loc[0,'T_SAND']
        SOL_TABLE_RECLASS.loc[i,'T_SILT'] = replaced_data.loc[0,'T_SILT']
        SOL_TABLE_RECLASS.loc[i,'T_CLAY'] = replaced_data.loc[0,'T_CLAY']
        SOL_TABLE_RECLASS.loc[i,'T_OC'] = replaced_data.loc[0,'T_OC']
        SOL_TABLE_RECLASS.loc[i,'T_PH_H2O'] = replaced_data.loc[0,'T_PH_H2O']
        SOL_TABLE_RECLASS.loc[i,'T_CACO3'] = replaced_data.loc[0,'T_CACO3']
        SOL_TABLE_RECLASS.loc[i,'T_ECE'] = replaced_data.loc[0,'T_ECE']
        SOL_TABLE_RECLASS.loc[i,'S_GRAVEL'] = replaced_data.loc[0,'S_GRAVEL']
        SOL_TABLE_RECLASS.loc[i,'S_SAND'] = replaced_data.loc[0,'S_SAND']
        SOL_TABLE_RECLASS.loc[i,'S_SILT'] = replaced_data.loc[0,'S_SILT']
        SOL_TABLE_RECLASS.loc[i,'S_CLAY'] = replaced_data.loc[0,'S_CLAY']
        SOL_TABLE_RECLASS.loc[i,'S_OC'] = replaced_data.loc[0,'S_OC']
        SOL_TABLE_RECLASS.loc[i,'S_CACO3'] = replaced_data.loc[0,'S_CACO3']
        SOL_TABLE_RECLASS.loc[i,'S_PH_H2O'] = replaced_data.loc[0,'S_PH_H2O']

# 计算有机质
top_soil_organic_matter = []
second_soil_organic_matter = []
for i in range(SOL_TABLE_RECLASS.shape[0]):
    if pd.isna(SOL_TABLE_RECLASS.loc[i,'T_OC']):
        top_soil_organic_matter.append(np.nan)
    else:
        top_soil_organic_matter.append(SOL_TABLE_RECLASS.loc[i,'T_OC']/0.58)

    if pd.isna(SOL_TABLE_RECLASS.loc[i,'S_OC']):
        second_soil_organic_matter.append(np.nan)
    else:
        second_soil_organic_matter.append(SOL_TABLE_RECLASS.loc[i,'S_OC']/0.58)

T_OC_IDX = SOL_TABLE_RECLASS.columns.get_loc('T_OC')
SOL_TABLE_RECLASS.insert(T_OC_IDX+1,'T_ORGANIC_MATTER',top_soil_organic_matter)
S_OC_IDX = SOL_TABLE_RECLASS.columns.get_loc('S_OC')
SOL_TABLE_RECLASS.insert(S_OC_IDX+1,'S_ORGANIC_MATTER',second_soil_organic_matter)

# 导出土壤重分类结果
SOL_TABLE_RECLASS.sort_values(by=['MU_GLOBAL'],ascending=True,inplace=True)
SOL_TABLE_RECLASS.to_excel(workspace+'SOL_TABLE_RECLASSIFIED.xlsx',index=None)

print("需要替换的MU_GLOBAL：",replace_mu_global)


GG
RK
ST
0.49
UR
WR
ATc
FLc
HSs
KSh
GLm
GRh
RGc
SCm
GYk
ARh
LPi
CLh
CMi
LVh
PHh
CHl
需要替换的MU_GLOBAL： ['11930', '11933']


In [40]:
# 初始化一个pandas dataframe
target_usersoil = pd.DataFrame(columns=usersoil.columns) #target_usersoil对应土壤属性数据表
temp_usersoil = pd.DataFrame(columns=usersoil.columns) #temp_usersoil对应临时存储的土壤属性数据，包括WR\RK\UR



# 计算target_usersoil各列数据：总列数=12+12*10+10+10 = 152列数据
OBJECTID = []
MUID = []
SEQN = []
SNAM = [] #土壤名称，填SU_SYM90名称
S5ID = [] 
CMPPCT = []
NLAYERS = [] # 土壤层数量：HWSD土壤为2层
HYDGRP = [] # 土壤水文分组A,B,C,D
SOL_ZMX = [] #土壤剖面最大深度，默认1000，HWSD里REF_DEPTH*10（SWAT单位：mm,HWSD单位cm）
ANION_EXCL = [] #阴离子交换孔隙度（默认0.5）
SOL_CRK = [] #土壤剖面潜在或最大裂隙体积（0.5）
TEXTURE = [] #!土层结构（根据SPAW的结果确定）
# ?TEXTURE代码：C=Clay;CL=Clay Loam;L=Loam;LS=Loamy Sand; Sa=Sand;SC = Sand Clay;SCL = Sandy Clay Loam; SL=Sandy Loam
# ?TEXTURE代码：Si=Silt;SiC=Silty Clay; SiCL = Silty Clay Loam;SiL=Silty Loam
# 第一层土壤-12列
SOL_Z1 = [] #第1层土壤深度：表层到第1层底部；HWSD前1层300mm
SOL_BD1 = [] #!土壤湿容重（g/cm^3; SPAW计算获得:Matric Bulk Density）
SOL_AWC1 = [] #!土壤可利用水量（SPAW计算获得:Avaliable Water）
SOL_K1 = [] #!饱和水力传导系数（mm/hr; SPAW计算获得：Sat. Hydraulic Cond.）
SOL_CBN1 = [] #有机碳含量（重量百分比 % w.t.; 对应于HWSD_DATA里面的T_OC)
CLAY1 = [] # 黏土比例（重量百分比 % w.t.;对应于HWSD_DATA中T_CLAY）
SILT1 = [] # 粉砂土比例（重量百分比 % w.t.;对应于HWSD_DATA中T_SILT）
SAND1 = [] # 沙土比例（重量百分比 % w.t.;对应于HWSD_DATA中T_SAND）
ROCK1 = [] # 砾石比例 （体积百分比 % vol;对应于HWSD_DATA中T_GRAVEL）
SOL_ALB1 = [] # !湿润土壤反照率（默认；0.01，可以通过经验公式计算）
USLE_K1 = [] # !USLE方程中的可蚀性因子（经验公式计算获得）
SOL_EC1 = [] # 电导率（dS/m; 对应于HWSD_DATA中 T_ECE)

# 第二层土壤-12列
SOL_Z2 = [] #第2层土壤深度：表层到第2层底部；HWSD前2层1000mm
SOL_BD2 = [] #!土壤湿容重（g/cm^3; SPAW计算获得:Matric Bulk Density）
SOL_AWC2 = [] #!土壤可利用水量（SPAW计算获得:Avaliable Water）
SOL_K2 = [] #!饱和水力传导系数（mm/hr; SPAW计算获得：Sat. Hydraulic Cond.）
SOL_CBN2 = [] #有机碳含量（重量百分比 % w.t.; 对应于HWSD_DATA里面的S_OC)
CLAY2 = [] # 黏土比例（重量百分比 % w.t.;对应于HWSD_DATA中S_CLAY）
SILT2 = [] # 粉砂土比例（重量百分比 % w.t.;对应于HWSD_DATA中S_SILT）
SAND2 = [] # 沙土比例（重量百分比 % w.t.;对应于HWSD_DATA中S_SAND）
ROCK2 = [] # 砾石比例 （体积百分比 % vol;对应于HWSD_DATA中S_GRAVEL）
SOL_ALB2 = [] # !湿润土壤反照率（默认；0.01，可以通过经验公式计算）
USLE_K2 = [] # !USLE方程中的可蚀性因子（经验公式计算获得）
SOL_EC2 = [] # 电导率（dS/m; 对应于HWSD_DATA中 S_ECE)

# 第三层土壤-12列 （HWSD v1.2只有2层，后面数据全部为0）
SOL_Z3 = []
SOL_BD3 = []
SOL_AWC3 = []
SOL_K3 = []
SOL_CBN3 = []
CLAY3 = []
SILT3 = []
SAND3 = []
ROCK3 = []
SOL_ALB3 = []
USLE_K3 = []
SOL_EC3 = []

# 第四层土壤-12列
SOL_Z4 = []
SOL_BD4 = []
SOL_AWC4 = []
SOL_K4 = []
SOL_CBN4 = []
CLAY4 = []
SILT4 = []
SAND4 = []
ROCK4 = []
SOL_ALB4 = []
USLE_K4 = []
SOL_EC4 = []

# 第五层土壤-12列
SOL_Z5 = []
SOL_BD5 = []
SOL_AWC5 = []
SOL_K5 = []
SOL_CBN5 = []
CLAY5 = []
SILT5 = []
SAND5 = []
ROCK5 = []
SOL_ALB5 = []
USLE_K5 = []
SOL_EC5 = []

# 第六层土壤-12列
SOL_Z6 = []
SOL_BD6 = []
SOL_AWC6 = []
SOL_K6 = []
SOL_CBN6 = []
CLAY6 = []
SILT6 = []
SAND6 = []
ROCK6 = []
SOL_ALB6 = []
USLE_K6 = []
SOL_EC6 = []

# 第七层土壤-12列
SOL_Z7 = []
SOL_BD7 = []
SOL_AWC7 = []
SOL_K7 = []
SOL_CBN7 = []
CLAY7 = []
SILT7 = []
SAND7 = []
ROCK7 = []
SOL_ALB7 = []
USLE_K7 = []
SOL_EC7 = []

# 第8层土壤-12列
SOL_Z8 = []
SOL_BD8 = []
SOL_AWC8 = []
SOL_K8 = []
SOL_CBN8 = []
CLAY8 = []
SILT8 = []
SAND8 = []
ROCK8 = []
SOL_ALB8 = []
USLE_K8 = []
SOL_EC8 = []

# 第九层土壤-12列
SOL_Z9 = []
SOL_BD9 = []
SOL_AWC9= []
SOL_K9 = []
SOL_CBN9 = []
CLAY9 = []
SILT9 = []
SAND9 = []
ROCK9 = []
SOL_ALB9 = []
USLE_K9 = []
SOL_EC9 = []

# 第十层土壤-12列
SOL_Z10 = []
SOL_BD10 = []
SOL_AWC10 = []
SOL_K10 = []
SOL_CBN10 = []
CLAY10 = []
SILT10 = []
SAND10 = []
ROCK10 = []
SOL_ALB10 = []
USLE_K10 = []
SOL_EC10 = []

# 碳酸盐-10列
SOL_CAL1 = [] # 第1层碳酸盐含量，对应T_CACO3
SOL_CAL2 = []  # 第2层碳酸盐含量，对应S_CACO3
SOL_CAL3 = []
SOL_CAL4 = []
SOL_CAL5 = []
SOL_CAL6 = []
SOL_CAL7 = []
SOL_CAL8 = []
SOL_CAL9 = []
SOL_CAL10 = []

# PH值-10列
SOL_PH1 = [] # 第1层酸碱度，对应T_PH_H20
SOL_PH2 = [] # 第2层酸碱度，对应S_PH_H20
SOL_PH3 = []# 其他为0
SOL_PH4 = []
SOL_PH5 = []
SOL_PH6 = []
SOL_PH7 = []
SOL_PH8 = []
SOL_PH9 = []
SOL_PH10 = []




for i in range(SOL_TABLE_RECLASS.shape[0]):
    ## 判断是否存在水体类型，如果是则是直接从usersoil中提取
    if SOL_TABLE_RECLASS.loc[i,'SU_SYM90'] == 'WR':
        temp_usersoil = pd.concat([temp_usersoil,usersoil[usersoil['SNAM']=='WATER']],axis=0,ignore_index=True)
        temp_usersoil.loc[temp_usersoil['SNAM']=='WATER','OBJECTID'] = SOL_TABLE_RECLASS.loc[i,'MU_GLOBAL']
    elif SOL_TABLE_RECLASS.loc[i,'SU_SYM90'] == 'UR':
        temp_usersoil = pd.concat([temp_usersoil,usersoil[usersoil['SNAM']=='URBAN LAND']],axis=0,ignore_index=True)
        temp_usersoil.loc[temp_usersoil['SNAM']=='URBAN LAND','OBJECTID'] = SOL_TABLE_RECLASS.loc[i,'MU_GLOBAL']
    elif SOL_TABLE_RECLASS.loc[i,'SU_SYM90'] == 'RK':
        temp_usersoil = pd.concat([temp_usersoil,usersoil[usersoil['SNAM']=='ROCK OUTCROP']],axis=0,ignore_index=True)
        temp_usersoil.loc[temp_usersoil['SNAM']=='ROCK OUTCROP','OBJECTID'] = SOL_TABLE_RECLASS.loc[i,'MU_GLOBAL']
    else:
        OBJECTID.append(SOL_TABLE_RECLASS.loc[i,'MU_GLOBAL'])
        MUID.append(0)
        SEQN.append(SOL_TABLE_RECLASS['SEQ'][i])
        SNAM.append(SOL_TABLE_RECLASS.loc[i,'SU_SYM90'])
        S5ID.append(0)
        CMPPCT.append(0)

        ANION_EXCL.append(0.5)
        SOL_CRK.append(0.5)
        TEXTURE.append(None) #后续由SPAW计算获得包括第一层和第二层，用‘-’隔开
        SOL_BD1.append(None) #后续由SPAW计算获得
        SOL_AWC1.append(None) #后续由SPAW计算获得
        SOL_K1.append(None) #后续由SPAW计算获得
        
    
        
        if SOL_TABLE_RECLASS.loc[i,'REF_DEPTH'] == 100:# 27 variables
            NLAYERS.append(2)
            SOL_ZMX.append(1000)
            SOL_Z1.append(300)

            SOL_CBN1.append(SOL_TABLE_RECLASS.loc[i,'T_OC'])
            CLAY1.append(SOL_TABLE_RECLASS.loc[i,'T_CLAY'])
            SILT1.append(SOL_TABLE_RECLASS.loc[i,'T_SILT'])
            SAND1.append(SOL_TABLE_RECLASS.loc[i,'T_SAND'])
            ROCK1.append(SOL_TABLE_RECLASS.loc[i,'T_GRAVEL'])
            SOL_ALB1.append(compute_sol_alb(SOL_TABLE_RECLASS.loc[i,'T_OC'])) 
            USLE_K1.append(compute_usle_k(
                SAND=SOL_TABLE_RECLASS.loc[i,'T_SAND'],
                SILT=SOL_TABLE_RECLASS.loc[i,'T_SILT'],
                CLAY=SOL_TABLE_RECLASS.loc[i,'T_CLAY'],
                OC=SOL_TABLE_RECLASS.loc[i,'T_OC'],
            ))
            SOL_EC1.append(SOL_TABLE_RECLASS.loc[i,'T_ECE'])
            SOL_CAL1.append(SOL_TABLE_RECLASS.loc[i,'T_CACO3'])
            SOL_PH1.append(SOL_TABLE_RECLASS.loc[i,'T_PH_H2O'])

            SOL_Z2.append(1000)
            SOL_BD2.append(None) #后续由SPAW计算获得
            SOL_AWC2.append(None) #后续由SPAW计算获得
            SOL_K2.append(None) #后续由SPAW计算获得
            SOL_CBN2.append(SOL_TABLE_RECLASS.loc[i,'S_OC'])
            CLAY2.append(SOL_TABLE_RECLASS.loc[i,'S_CLAY'])
            SILT2.append(SOL_TABLE_RECLASS.loc[i,'S_SILT'])
            SAND2.append(SOL_TABLE_RECLASS.loc[i,'S_SAND'])
            ROCK2.append(SOL_TABLE_RECLASS.loc[i,'S_GRAVEL'])
            SOL_ALB2.append(compute_sol_alb(SOL_TABLE_RECLASS.loc[i,'S_OC']))
            USLE_K2.append(compute_usle_k(
                SAND = SOL_TABLE_RECLASS.loc[i,'S_SAND'],
                SILT = SOL_TABLE_RECLASS.loc[i,'S_SILT'],
                CLAY = SOL_TABLE_RECLASS.loc[i,'S_CLAY'],
                OC = SOL_TABLE_RECLASS.loc[i,'S_OC'],
            ))
            SOL_EC2.append(SOL_TABLE_RECLASS.loc[i,'S_ECE'])
            SOL_CAL2.append(SOL_TABLE_RECLASS.loc[i,'S_CACO3'])
            SOL_PH2.append(SOL_TABLE_RECLASS.loc[i,'S_PH_H2O'])
            

        elif SOL_TABLE_RECLASS.loc[i,'REF_DEPTH'] == 30:
            NLAYERS.append(1)
            SOL_ZMX.append(300)
            SOL_Z1.append(300)

            SOL_CBN1.append(SOL_TABLE_RECLASS.loc[i,'T_OC'])
            CLAY1.append(SOL_TABLE_RECLASS.loc[i,'T_CLAY'])
            SILT1.append(SOL_TABLE_RECLASS.loc[i,'T_SILT'])
            SAND1.append(SOL_TABLE_RECLASS.loc[i,'T_SAND'])
            ROCK1.append(SOL_TABLE_RECLASS.loc[i,'T_GRAVEL'])
            SOL_ALB1.append(compute_sol_alb(SOL_TABLE_RECLASS.loc[i,'T_OC']))
            USLE_K1.append(compute_usle_k(
                SAND = SOL_TABLE_RECLASS.loc[i,'T_SAND'],
                SILT = SOL_TABLE_RECLASS.loc[i,'T_SILT'],
                CLAY = SOL_TABLE_RECLASS.loc[i,'T_CLAY'],
                OC = SOL_TABLE_RECLASS.loc[i,'T_OC'],
            ))
            SOL_EC1.append(SOL_TABLE_RECLASS.loc[i,'T_ECE'])
            SOL_CAL1.append(SOL_TABLE_RECLASS.loc[i,'T_CACO3'])
            SOL_PH1.append(SOL_TABLE_RECLASS.loc[i,'T_PH_H2O'])

            # 第二层开始全部为0
            SOL_Z2.append(0)
            SOL_BD2.append(0)
            SOL_AWC2.append(0)
            SOL_K2.append(0)
            SOL_CBN2.append(0)
            CLAY2.append(0)
            SILT2.append(0)
            SAND2.append(0)
            ROCK2.append(0)
            SOL_ALB2.append(0)
            USLE_K2.append(0)
            SOL_EC2.append(0)
            SOL_CAL2.append(0)
            SOL_PH2.append(0)

        HYDGRP.append(compute_hydgrp(
            T_SAND=SOL_TABLE_RECLASS.loc[i,'T_SAND'],
            S_SAND=SOL_TABLE_RECLASS.loc[i,'S_SAND']
        ))

        SOL_Z3.append(0)
        SOL_BD3.append(0)
        SOL_AWC3.append(0)
        SOL_K3.append(0)
        SOL_CBN3.append(0)
        CLAY3.append(0)
        SILT3.append(0)
        SAND3.append(0)
        ROCK3.append(0)
        SOL_ALB3.append(0)
        USLE_K3.append(0)
        SOL_EC3.append(0)
        SOL_CAL3.append(0)
        SOL_PH3.append(0)

        SOL_Z4.append(0)
        SOL_BD4.append(0)
        SOL_AWC4.append(0)
        SOL_K4.append(0)
        SOL_CBN4.append(0)
        CLAY4.append(0)
        SILT4.append(0)
        SAND4.append(0)
        ROCK4.append(0)
        SOL_ALB4.append(0)
        USLE_K4.append(0)
        SOL_EC4.append(0)
        SOL_CAL4.append(0)
        SOL_PH4.append(0)

        SOL_Z5.append(0)
        SOL_BD5.append(0)
        SOL_AWC5.append(0)
        SOL_K5.append(0)
        SOL_CBN5.append(0)
        CLAY5.append(0)
        SILT5.append(0)
        SAND5.append(0)
        ROCK5.append(0)
        SOL_ALB5.append(0)
        USLE_K5.append(0)
        SOL_EC5.append(0)
        SOL_CAL5.append(0)
        SOL_PH5.append(0)

        SOL_Z6.append(0)
        SOL_BD6.append(0)
        SOL_AWC6.append(0)
        SOL_K6.append(0)
        SOL_CBN6.append(0)
        CLAY6.append(0)
        SILT6.append(0)
        SAND6.append(0)
        ROCK6.append(0)
        SOL_ALB6.append(0)
        USLE_K6.append(0)
        SOL_EC6.append(0)
        SOL_CAL6.append(0)
        SOL_PH6.append(0)

        SOL_Z7.append(0)
        SOL_BD7.append(0)
        SOL_AWC7.append(0)
        SOL_K7.append(0)
        SOL_CBN7.append(0)
        CLAY7.append(0)
        SILT7.append(0)
        SAND7.append(0)
        ROCK7.append(0)
        SOL_ALB7.append(0)
        USLE_K7.append(0)
        SOL_EC7.append(0)
        SOL_CAL7.append(0)
        SOL_PH7.append(0)

        SOL_Z8.append(0)
        SOL_BD8.append(0)
        SOL_AWC8.append(0)
        SOL_K8.append(0)
        SOL_CBN8.append(0)
        CLAY8.append(0)
        SILT8.append(0)
        SAND8.append(0)
        ROCK8.append(0)
        SOL_ALB8.append(0)
        USLE_K8.append(0)
        SOL_EC8.append(0)
        SOL_CAL8.append(0)
        SOL_PH8.append(0)

        SOL_Z9.append(0)
        SOL_BD9.append(0)
        SOL_AWC9.append(0)
        SOL_K9.append(0)
        SOL_CBN9.append(0)
        CLAY9.append(0)
        SILT9.append(0)
        SAND9.append(0)
        ROCK9.append(0)
        SOL_ALB9.append(0)
        USLE_K9.append(0)
        SOL_EC9.append(0)
        SOL_CAL9.append(0)
        SOL_PH9.append(0)

        SOL_Z10.append(0)
        SOL_BD10.append(0)
        SOL_AWC10.append(0)
        SOL_K10.append(0)
        SOL_CBN10.append(0)
        CLAY10.append(0)
        SILT10.append(0)
        SAND10.append(0)
        ROCK10.append(0)
        SOL_ALB10.append(0)
        USLE_K10.append(0)
        SOL_EC10.append(0)
        SOL_CAL10.append(0)
        SOL_PH10.append(0)

target_usersoil['OBJECTID'] = OBJECTID
target_usersoil['MUID'] = MUID
target_usersoil['SEQN'] = SEQN
target_usersoil['SNAM'] = SNAM
target_usersoil['S5ID'] = S5ID
target_usersoil['CMPPCT'] = CMPPCT
target_usersoil['NLAYERS'] = NLAYERS
target_usersoil['HYDGRP'] = HYDGRP
target_usersoil['SOL_ZMX'] = SOL_ZMX
target_usersoil['ANION_EXCL'] = ANION_EXCL
target_usersoil['SOL_CRK'] = SOL_CRK
target_usersoil['TEXTURE'] = TEXTURE
target_usersoil['SOL_Z1'] = SOL_Z1
target_usersoil['SOL_BD1'] = SOL_BD1
target_usersoil['SOL_AWC1'] = SOL_AWC1
target_usersoil['SOL_K1'] = SOL_K1
target_usersoil['SOL_CBN1'] = SOL_CBN1
target_usersoil['CLAY1'] = CLAY1
target_usersoil['SILT1'] = SILT1
target_usersoil['SAND1'] = SAND1
target_usersoil['ROCK1'] = ROCK1
target_usersoil['SOL_ALB1'] = SOL_ALB1
target_usersoil['USLE_K1'] = USLE_K1
target_usersoil['SOL_EC1'] = SOL_EC1
target_usersoil['SOL_Z2'] = SOL_Z2
target_usersoil['SOL_BD2'] = SOL_BD2
target_usersoil['SOL_AWC2'] = SOL_AWC2
target_usersoil['SOL_K2'] = SOL_K2
target_usersoil['SOL_CBN2'] = SOL_CBN2
target_usersoil['CLAY2'] = CLAY2
target_usersoil['SILT2'] = SILT2
target_usersoil['SAND2'] = SAND2
target_usersoil['ROCK2'] = ROCK2
target_usersoil['SOL_ALB2'] = SOL_ALB2
target_usersoil['USLE_K2'] = USLE_K2
target_usersoil['SOL_EC2'] = SOL_EC2
target_usersoil['SOL_Z3'] = SOL_Z3
target_usersoil['SOL_BD3'] = SOL_BD3
target_usersoil['SOL_AWC3'] = SOL_AWC3
target_usersoil['SOL_K3'] = SOL_K3
target_usersoil['SOL_CBN3'] = SOL_CBN3
target_usersoil['CLAY3'] = CLAY3
target_usersoil['SILT3'] = SILT3
target_usersoil['SAND3'] = SAND3
target_usersoil['ROCK3'] = ROCK3
target_usersoil['SOL_ALB3'] = SOL_ALB3
target_usersoil['USLE_K3'] = USLE_K3
target_usersoil['SOL_EC3'] = SOL_EC3
target_usersoil['SOL_Z4'] = SOL_Z4
target_usersoil['SOL_BD4'] = SOL_BD4
target_usersoil['SOL_AWC4'] = SOL_AWC4
target_usersoil['SOL_K4'] = SOL_K4
target_usersoil['SOL_CBN4'] = SOL_CBN4
target_usersoil['CLAY4'] = CLAY4
target_usersoil['SILT4'] = SILT4
target_usersoil['SAND4'] = SAND4
target_usersoil['ROCK4'] = ROCK4
target_usersoil['SOL_ALB4'] = SOL_ALB4
target_usersoil['USLE_K4'] = USLE_K4
target_usersoil['SOL_EC4'] = SOL_EC4
target_usersoil['SOL_Z5'] = SOL_Z5
target_usersoil['SOL_BD5'] = SOL_BD5
target_usersoil['SOL_AWC5'] = SOL_AWC5
target_usersoil['SOL_K5'] = SOL_K5
target_usersoil['SOL_CBN5'] = SOL_CBN5
target_usersoil['CLAY5'] = CLAY5
target_usersoil['SILT5'] = SILT5
target_usersoil['SAND5'] = SAND5
target_usersoil['ROCK5'] = ROCK5
target_usersoil['SOL_ALB5'] = SOL_ALB5
target_usersoil['USLE_K5'] = USLE_K5
target_usersoil['SOL_EC5'] = SOL_EC5
target_usersoil['SOL_Z6'] = SOL_Z6
target_usersoil['SOL_BD6'] = SOL_BD6
target_usersoil['SOL_AWC6'] = SOL_AWC6
target_usersoil['SOL_K6'] = SOL_K6
target_usersoil['SOL_CBN6'] = SOL_CBN6
target_usersoil['CLAY6'] = CLAY6
target_usersoil['SILT6'] = SILT6
target_usersoil['SAND6'] = SAND6
target_usersoil['ROCK6'] = ROCK6
target_usersoil['SOL_ALB6'] = SOL_ALB6
target_usersoil['USLE_K6'] = USLE_K6
target_usersoil['SOL_EC6'] = SOL_EC6
target_usersoil['SOL_Z7'] = SOL_Z7
target_usersoil['SOL_BD7'] = SOL_BD7
target_usersoil['SOL_AWC7'] = SOL_AWC7
target_usersoil['SOL_K7'] = SOL_K7
target_usersoil['SOL_CBN7'] = SOL_CBN7
target_usersoil['CLAY7'] = CLAY7
target_usersoil['SILT7'] = SILT7
target_usersoil['SAND7'] = SAND7
target_usersoil['ROCK7'] = ROCK7
target_usersoil['SOL_ALB7'] = SOL_ALB7
target_usersoil['USLE_K7'] = USLE_K7
target_usersoil['SOL_EC7'] = SOL_EC7
target_usersoil['SOL_Z8'] = SOL_Z8
target_usersoil['SOL_BD8'] = SOL_BD8
target_usersoil['SOL_AWC8'] = SOL_AWC8
target_usersoil['SOL_K8'] = SOL_K8
target_usersoil['SOL_CBN8'] = SOL_CBN8
target_usersoil['CLAY8'] = CLAY8
target_usersoil['SILT8'] = SILT8
target_usersoil['SAND8'] = SAND8
target_usersoil['ROCK8'] = ROCK8
target_usersoil['SOL_ALB8'] = SOL_ALB8
target_usersoil['USLE_K8'] = USLE_K8
target_usersoil['SOL_EC8'] = SOL_EC8
target_usersoil['SOL_Z9'] = SOL_Z9
target_usersoil['SOL_BD9'] = SOL_BD9
target_usersoil['SOL_AWC9'] = SOL_AWC9
target_usersoil['SOL_K9'] = SOL_K9
target_usersoil['SOL_CBN9'] = SOL_CBN9
target_usersoil['CLAY9'] = CLAY9
target_usersoil['SILT9'] = SILT9
target_usersoil['SAND9'] = SAND9
target_usersoil['ROCK9'] = ROCK9
target_usersoil['SOL_ALB9'] = SOL_ALB9
target_usersoil['USLE_K9'] = USLE_K9
target_usersoil['SOL_EC9'] = SOL_EC9
target_usersoil['SOL_Z10'] = SOL_Z10
target_usersoil['SOL_BD10'] = SOL_BD10
target_usersoil['SOL_AWC10'] = SOL_AWC10
target_usersoil['SOL_K10'] = SOL_K10
target_usersoil['SOL_CBN10'] = SOL_CBN10
target_usersoil['CLAY10'] = CLAY10
target_usersoil['SILT10'] = SILT10
target_usersoil['SAND10'] = SAND10
target_usersoil['ROCK10'] = ROCK10
target_usersoil['SOL_ALB10'] = SOL_ALB10
target_usersoil['USLE_K10'] = USLE_K10
target_usersoil['SOL_EC10'] = SOL_EC10
target_usersoil['SOL_CAL1'] = SOL_CAL1
target_usersoil['SOL_CAL2'] = SOL_CAL2
target_usersoil['SOL_CAL3'] = SOL_CAL3
target_usersoil['SOL_CAL4'] = SOL_CAL4
target_usersoil['SOL_CAL5'] = SOL_CAL5
target_usersoil['SOL_CAL6'] = SOL_CAL6
target_usersoil['SOL_CAL7'] = SOL_CAL7
target_usersoil['SOL_CAL8'] = SOL_CAL8
target_usersoil['SOL_CAL9'] = SOL_CAL9
target_usersoil['SOL_CAL10'] = SOL_CAL10
target_usersoil['SOL_PH1'] = SOL_PH1
target_usersoil['SOL_PH2'] = SOL_PH2
target_usersoil['SOL_PH3'] = SOL_PH3
target_usersoil['SOL_PH4'] = SOL_PH4
target_usersoil['SOL_PH5'] = SOL_PH5
target_usersoil['SOL_PH6'] = SOL_PH6
target_usersoil['SOL_PH7'] = SOL_PH7
target_usersoil['SOL_PH8'] = SOL_PH8
target_usersoil['SOL_PH9'] = SOL_PH9
target_usersoil['SOL_PH10'] = SOL_PH10

temp_usersoil.reset_index(drop=True, inplace=True)
target_usersoil = pd.concat([target_usersoil, temp_usersoil], axis=0, ignore_index=True)

target_usersoil.reset_index(drop=True, inplace=True)
target_usersoil.sort_values(by=['OBJECTID'], inplace=True)

# 读取已有target_usersoil数据
full_target_usersoil = pd.read_csv('../data/target_usersoil_full.csv')
OBJECTID_full = full_target_usersoil["OBJECTID"].to_list()

for object_id in target_usersoil["OBJECTID"]:
    if object_id in OBJECTID_full:
        row_to_replace = full_target_usersoil[full_target_usersoil["OBJECTID"]==object_id].iloc[0]
        # print(row_to_replace.shape[1],target_usersoil.shape[1])
        target_usersoil.loc[target_usersoil["OBJECTID"]==object_id,:] = row_to_replace.values

    

target_usersoil.to_excel(workspace+'target_usersoil_spaw.xlsx', index=None)

lookup_soil = pd.DataFrame(
    {'SOIL_ID':target_usersoil['OBJECTID'],'SNAM':target_usersoil['SNAM']}
)
lookup_soil.to_csv(workspace+'target_soil.csv',index=None)

SAND=56.0,SILT=38.0,CLAY=6.0,OC=1.41
f_csand=0.20004139738661672,f_cl_si=0.9569720993316607,f_orgc=0.82841359159047,f_hisand=0.9968115514479111
T_X=10.847711361614275; S_X=nan; min_x=10.847711361614275
SAND=24.0,SILT=44.0,CLAY=32.0,OC=0.49
f_csand=0.20961325114840934,f_cl_si=0.8487733309294013,f_orgc=0.9880055492914753,f_hisand=0.9999963657833831
SAND=24.0,SILT=40.0,CLAY=36.0,OC=0.27
f_csand=0.20751861897876764,f_cl_si=0.8248479886127263,f_orgc=0.9964240865261892,f_hisand=0.9999963657833831
T_X=2.9968988247353194; S_X=2.9968988247353194; min_x=2.9968988247353194
SAND=29.0,SILT=50.0,CLAY=21.0,OC=1.12
f_csand=0.20732858514426067,f_cl_si=0.9001471207799131,f_orgc=0.8937738529172452,f_hisand=0.9999893311309771
SAND=34.0,SILT=45.0,CLAY=21.0,OC=0.82
f_csand=0.2025007294633578,f_cl_si=0.8914573570107817,f_orgc=0.9543731428432063,f_hisand=0.9999688356239708
T_X=3.933188783854704; S_X=4.980829636067284; min_x=3.933188783854704
SAND=34.0,SILT=48.0,CLAY=18.0,OC=0.6
f_csand=0.20324690577841284,f_c

C:\Users\ZJY\AppData\Local\Temp\ipykernel_84756\1777394266.py:198: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  temp_usersoil = pd.concat([temp_usersoil,usersoil[usersoil['SNAM']=='ROCK OUTCROP']],axis=0,ignore_index=True)
C:\Users\ZJY\AppData\Local\Temp\ipykernel_84756\1777394266.py:575: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  target_usersoil = pd.concat([target_usersoil, temp_usersoil], axis=0, ignore_index=True)


识别出需要新补充的Object(土壤类型)，使用SPAW软件进行补充，其中所需要的其他信息用[extract_hwsd_data.py](extract_hwsd_data.py)从HWSD数据库中提取

In [ ]:
target_objectid_list = target_usersoil["OBJECTID"].to_list()
non_finished_object = list(set(target_objectid_list)-set(OBJECTID_full))
non_finished_object

hwsd = pd.read_excel("../data/HWSD_DATA.xlsx", sheet_name='HWSD_DATA')


inserted_columns = ['T_ECE','T_OM','S_ECE','S_OM']
inserted_index = [13,13,27,27]
target_usersoil = target_usersoil.drop(columns=inserted_columns)
for column,idx in zip(inserted_columns,inserted_index):
    target_usersoil.insert(idx,column=column,value=[-10 for i in target_usersoil['OBJECTID']])

for object_id in target_usersoil["OBJECTID"].to_list():
    top_organic_matter = hwsd.loc[hwsd['MU_GLOBAL']==object_id,'T_OC'].values[0]
    second_organic_matter = hwsd.loc[hwsd['MU_GLOBAL']==object_id,'S_OC'].values[0]
    top_ece = hwsd.loc[hwsd['MU_GLOBAL']==object_id,'T_ECE'].values[0]
    second_ece = hwsd.loc[hwsd['MU_GLOBAL']==object_id,'S_ECE'].values[0]

    target_usersoil.loc[target_usersoil['OBJECTID']==object_id,'T_ECE'] = top_ece
    target_usersoil.loc[target_usersoil['OBJECTID']==object_id,'S_ECE'] = second_ece
    
    if np.isnan(top_organic_matter):
        target_usersoil.loc[target_usersoil['OBJECTID']==object_id,'T_OM'] = top_organic_matter
    else:
        target_usersoil.loc[target_usersoil['OBJECTID']==object_id,'T_OM'] = top_organic_matter/0.58

    if np.isnan(second_organic_matter):
        target_usersoil.loc[target_usersoil['OBJECTID']==object_id,'S_OM'] = second_organic_matter
    else:
        target_usersoil.loc[target_usersoil['OBJECTID']==object_id,'S_OM'] = second_organic_matter/0.58

    

target_usersoil.to_excel(workspace+'target_usersoil_spaw.xlsx', index=None)


In [28]:
target_usersoil = pd.read_excel(workspace+'target_usersoil_spaw - edited.xlsx',sheet_name='Sheet1')
target_usersoil = target_usersoil.drop(columns=inserted_columns)
target_usersoil.to_csv(workspace+'target_usersoil.csv',index=None)

not_deal_soils = ['DS','ST','GG','ND','FP','IS']
# drop rows where SNAM in not_deal_soils
target_usersoil = target_usersoil[~target_usersoil['SNAM'].isin(not_deal_soils)]
full_target_usersoil = full_target_usersoil[~full_target_usersoil['SNAM'].isin(not_deal_soils)]

for object_id in target_usersoil["OBJECTID"].to_list():
    if object_id not in OBJECTID_full and object_id not in replace_mu_global:
        # insert the row corresponding to object_id from target_usersoil into full_target_usersoil
        row_to_insert = target_usersoil.loc[target_usersoil['OBJECTID']==object_id]
        full_target_usersoil = pd.concat([full_target_usersoil, row_to_insert], ignore_index=True)
        

full_target_usersoil.to_csv('../data/target_usersoil_full.csv',index=None)


